<a href="https://colab.research.google.com/github/masp42/CA_AI/blob/main/CA_DAvid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics as sts
import timeit
from timeit import Timer

In [38]:
df = pd.read_csv("https://raw.githubusercontent.com/masp42/CA_AI/main/Graph_Fig1.csv")  


In [39]:
df.head(15)

,Node1,Node2,Distance
0,A,B,5
1,A,C,3
2,B,C,2
3,C,D,7
4,D,A,2
5,B,G,1
6,B,E,3
7,G,E,1
8,C,E,7
9,E,D,2


# Dijkstra’s algorithm

In [48]:
class Dijkstra():

    def __init__(self):
           self.Graph_nodes = dict()

    def add(self, graph):
            self.Graph_nodes = graph

    def dijkstraAlgo(self, start_node, stop_node):

            path = {}
            adj_node = {}
            queue = []
            graph = self.Graph_nodes
            for node in graph:
                path[node] = float("inf")
                adj_node[node] = None
                queue.append(node)
                
            path[start_node] = 0
            while queue:
                # find min distance which wasn't marked as current
                key_min = queue[0]
                min_val = path[key_min]
                for n in range(1, len(queue)):
                    if path[queue[n]] < min_val:
                        key_min = queue[n]  
                        min_val = path[key_min]
                cur = key_min
                queue.remove(cur)
                
                for i in graph[cur]:
                    alternate = graph[cur][i] + path[cur]
                    if path[i] > alternate:
                        path[i] = alternate
                        adj_node[i] = cur
                        
                        
            print(stop_node, end = '<-')
            while True:
                stop_node = adj_node[stop_node]
                if stop_node is None:
                    print("")
                    break
                print(stop_node, end='<-')


In [49]:
NodeDistance_Dijkstra = df.groupby('Node1').apply(lambda x: dict(zip(x['Node2'],x['Distance']))).to_dict()
NodeDistance_Dijkstra

{'A': {'B': 5, 'C': 3},
 'B': {'C': 2, 'E': 3, 'G': 1},
 'C': {'D': 7, 'E': 7},
 'D': {'A': 2, 'F': 6},
 'E': {'D': 2, 'F': 1},
 'F': {'F': 0},
 'G': {'E': 1}}

In [50]:
def runDijkstra():
    dijkstra = Dijkstra()
    dijkstra.add(NodeDistance_Dijkstra)
    dijkstra.dijkstraAlgo("A", "F")

runDijkstra()

F<-E<-G<-B<-A<-


# A* Algorithm

In [26]:
class AStar ():

    def __init__(self):
           self.Graph_nodes = dict()

    def add(self, graph):
            self.Graph_nodes = graph

    def aStarAlgo(self, start_node, stop_node):
            
            open_set = set(start_node) 
            closed_set = set()
            g = {} #store distance from starting node
            parents = {}# parents contains an adjacency map of all nodes
    
            #ditance of starting node from itself is zero
            g[start_node] = 0
            #start_node is root node i.e it has no parent nodes
            #so start_node is set to its own parent node
            parents[start_node] = start_node
            
            
            while len(open_set) > 0:
                n = None
    
                #node with lowest f() is found
                for v in open_set:
                    if n == None or g[v] < g[n]:
                        n = v
                
                        
                if n == stop_node or self.Graph_nodes[n] == None:
                    pass
                else:                    
                    for (m, weight) in self.get_neighbors(n):
                        #nodes 'm' not in first and last set are added to first
                        #n is set its parent
                        if m not in open_set and m not in closed_set:
                            open_set.add(m)
                            parents[m] = n
                            g[m] = g[n] + weight
                            
        
                        #for each node m,compare its distance from start i.e g(m) to the
                        #from start through n node
                        else:
                            if g[m] > g[n] + weight:
                                #update g(m)
                                g[m] = g[n] + weight
                                #change parent of m to n
                                parents[m] = n
                                
                                #if m in closed set,remove and add to open
                                if m in closed_set:
                                    closed_set.remove(m)
                                    open_set.add(m)
    
                if n == None:
                    print('Path does not exist!')
                    return None
    
                # if the current node is the stop_node
                # then we begin reconstructin the path from it to the start_node
                if n == stop_node:
                    path = []
    
                    while parents[n] != n:
                        path.append(n)
                        n = parents[n]
    
                    path.append(start_node)
    
                    path.reverse()
    
                    print('Path found: {}'.format(path))
                    return path
    
    
                # remove n from the open_list, and add it to closed_list
                # because all of his neighbors were inspected
                open_set.remove(n)
                closed_set.add(n)
    
            print('Path does not exist!')
            return None

#==================================================================================

    #define fuction to return neighbor and its distance
    #from the passed node
    def get_neighbors(self, v):
        if v in self.Graph_nodes:
            return self.Graph_nodes[v]
        else:
            return None
    #for simplicity we ll consider heuristic distances given
    #and this function returns heuristic distance for all nodes
    def heuristic(self, n):
            H_dist = {
                'A': 11,
                'B': 6,
                'C': 99,
                'D': 1,
                'E': 7,
                'G': 0,
                
            }
    
            return H_dist[n]




In [27]:
NodeDistance = df.groupby('Node1')[['Node2', 'Distance']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
NodeDistance

{'A': [('B', 5), ('C', 3)],
 'B': [('C', 2), ('G', 1), ('E', 3)],
 'C': [('D', 7), ('E', 7)],
 'D': [('A', 2), ('F', 6)],
 'E': [('D', 2), ('F', 1)],
 'G': [('E', 1)]}

In [45]:
def runAStar():
    astar = AStar()
    astar.add(NodeDistance)
    astar.aStarAlgo("A","F")

runAStar()

Path found: ['A', 'B', 'G', 'E', 'F']


# Dijkstra vs A*

In [54]:
print("Dijkstra", Timer("runDijkstra()", "from __main__ import runDijkstra").timeit(number=1))
print("A*", Timer("runAStar()", "from __main__ import runAStar").timeit(number=1))

F<-E<-G<-B<-A<-
Dijkstra 0.0008465089995297603
Path found: ['A', 'B', 'G', 'E', 'F']
A* 0.00013790299999527633
